## パイプライン

### パイプラインとは
#### 変換器(特徴量の生成と選択)と予測気を直列に繋げ，一連の処理をまとめて実行できるようにする仕組みのことをパイプラインと言う．
### メリット
1. いくつかの前処理を実行している場合でも，fit や predict をパイプラインに対して1度だけ呼ぶだけで良い
2. パイプラインに対して，グリッドサーチを行う際，パラメータを1度に指定できる．fit を読んだ後の変換器をキャッシュでき，性能向上が期待できる．
3. transformer と estimatorに同じデータが使われるため，交差検証の際にデータがリークされることを防げる

### 実装

1. oputuna によるパラメータ種類と範囲の選択
2. クロスバリデーションをしながらパラメータチューニング
3. 評価指標の平均値を計算
4. パイプライン構築
    - 前処理は，標準化，主成分分析による次元削除
    - LightGBMはScikit-Learn APIを使用

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import pickle

from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import SelectFromModel


# 変換器
from sklearn.preprocessing import StandardScaler

# 予測器
import lightgbm as lgb
from lightgbm.basic import param_dict_to_str

# パイプライン
from sklearn.pipeline import Pipeline

# パラメーター調整
import optuna
import optuna.integration.lightgbm as lgb_tune
from lightgbm import early_stopping

# 評価関数
from sklearn.metrics import mean_squared_error, r2_score

# ベイズ最適化
import optuna.integration.lightgbm as ilgbm

### データの読み込み

In [6]:
df = pd.read_csv('../Data-science//data//boston.csv')

#X = df.drop("house prices", axis=1)
X= df[['INDUS', 'RM', 'TAX', 'PTRATIO', 'LSTAT']]
y = df['house prices']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=53)

X_train = X_train.values
X_test = X_test.values

y_train = y_train.values
y_test = y_test.values

### パラメータチューニング

Optunaによるベイズ最適化

In [7]:
# 標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train_sc = sc.transform(X_train)
X_test_sc = sc.transform(X_test)

## LightGBMのパラメータ

| Sckikit-Learn API | Training API     | 型 (スケール)  | 範囲     | デフォルト値 | 内容                                                                                                        |
|-------------------|------------------|----------------|----------|--------------|-------------------------------------------------------------------------------------------------------------|
| reg_alpha         | lambda_l1        | float (log)    | 0-∞      | 0            | L1正則化項の係数(小さいほど過学習寄り)                                                                      |
| reg_lambda        | lambda_l2        | float (log)    | 0-∞      | 0            | L2正則化項の係数(小さいほど過学習寄り)                                                                      |
| num_leaves        | num_leaves       | int (linear)   | 2-131072 | 31           | 1本の木の最大葉枚数(大きいほど過学習寄り)                                                                   |
| colsample _bytree | feature_fraction | float (linear) | 0-1      | 1.0          | 各決定木においてランダムに抽出される列の割合(大きいほど過学習寄り)                                          |
| subsample         | bagging_fraction | float (linear) | 0-1      | 1.0          | 各決定木においてランダムに抽出される標本の割合(大きいほど過学習寄り)                                        |
| subsample_freq    | bagging_freq     | int (linear)   | 0-∞      | 0            | ここで指定したイテレーション毎にバギング実施(大きいほど過学習寄りだが、0のときバギング非実施となるので注意) |
| min_child_samples | min_data_in_leaf | int (linear)   | 0-∞      | 20           | 1枚の葉に含まれる最小データ数(小さいほど過学習寄り)                                                         |

In [16]:
model = lgb.LGBMRegressor()

param_distributions = {
    'n_estimators':optuna.distributions.IntLogUniformDistribution(1e+0, 1e+3, 1),
    'max_depth': optuna.distributions.IntUniformDistribution(1, 20, 1),
    'reg_alpha': optuna.distributions.IntLogUniformDistribution(1e+0, 10.0, 0.1),
    'reg_lambda': optuna.distributions.IntLogUniformDistribution(1e+0, 10.0, 0.1),
    #'subsample_freq':optuna.distributions.IntUniformDistribution(1, 200, 5),
    'min_child_samples':optuna.distributions.IntUniformDistribution(1, 100, 5),
    #'bagging_freq':optuna.distributions.IntUniformDistribution(1, 10, 1),
    'num_leaves': optuna.distributions.IntUniformDistribution(100, 200, 5),
}

# クロスバリデーションによる、ハイパーパラメータの探索設定.cvのデフォルト設定値は5
optuna_search = optuna.integration.OptunaSearchCV(
    model,
    param_distributions,
    cv=7,
    #enable_pruning=True,
    n_jobs=-1,
    n_trials=100,
    random_state=42,
    scoring='neg_root_mean_squared_error'
)


/tmp/ipykernel_431/450061309.py:15: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(


In [17]:
# 探索の実行
optuna_search.fit(X_train_sc, y_train)

[I 2022-01-09 15:11:34,469] A new study created in memory with name: no-name-36e52f27-f2c9-4d50-bce3-f36d68c7fc0e
/home/bi18056/ml/venv/lib/python3.8/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-01-09 15:11:34,871] Trial 5 finished with value: -8.117335493556194 and parameters: {'n_estimators': 2, 'max_depth': 11, 'reg_alpha': 9, 'reg_lambda': 4, 'min_child_samples': 66, 'num_leaves': 170}. Best is trial 5 with value: -8.117335493556194.
[I 2022-01-09 15:11:35,216] Trial 4 finished with value: -5.578485870123208 and parameters: {'n_estimators': 19, 'max_depth': 3, 'reg_alpha': 10, 'reg_lambda': 1, 'min_child_samples': 81, 'num_leaves': 120}. Best is trial 4 with value: -5.578485870123208.
[I 2022-01-09 15:11:35,286] Trial 6 finished with value: -6.562186646837054 and parameters: {'n_estimators': 5, 'max_de

OptunaSearchCV(cv=7, estimator=LGBMRegressor(), n_jobs=-1, n_trials=100,
               param_distributions={'max_depth': IntUniformDistribution(high=20, low=1, step=1),
                                    'min_child_samples': IntUniformDistribution(high=96, low=1, step=5),
                                    'n_estimators': IntLogUniformDistribution(high=1000.0, low=1.0, step=1),
                                    'num_leaves': IntUniformDistribution(high=200, low=100, step=5),
                                    'reg_alpha': IntLogUniformDistribution(high=10.0, low=1.0, step=0.1),
                                    'reg_lambda': IntLogUniformDistribution(high=10.0, low=1.0, step=0.1)},
               random_state=42, scoring='neg_root_mean_squared_error')

In [18]:
optuna_search.best_score_

-3.5047325810799896

In [19]:
best_params = optuna_search.best_params_
best_params

{'n_estimators': 310,
 'max_depth': 5,
 'reg_alpha': 4,
 'reg_lambda': 3,
 'min_child_samples': 1,
 'num_leaves': 155}

In [24]:
print(optuna_search.score(X_test_sc, y_test))

-4.510549770961713


In [ ]:
y_pred = optuna_search.predict(X_test_sc)


### 前処理

#### 標準化

In [ ]:
scaler = StandardScaler()

#### 特徴量選択

分類器のfeature_importances_に基づき、重要度の高い特徴だけ残す．
分類気にはランダムフォレストを使う．

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.feature_selection import SelectFromModel

In [ ]:
rfr = RFR(n_estimators=100, n_jobs=-1)
fr = SelectFromModel(rfr)

#### 次元削除

次元削減にはPCAを使う．

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

In [ ]:
# 推定器
regressor = lgb.LGBMRegressor(param_dict_to_str(best_params))

estimators = [
    ('StandScaler', scaler),
    #('feature_selection', fr),
    ('pca', pca),
    ('estimator', regressor)
]

### パイプライン構築

In [ ]:
pipe = Pipeline(steps=estimators)

In [ ]:
# 学習
pipe.fit(X_train, y_train)

# 予測
pred = pipe.predict(X_test)

In [ ]:
pipe.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('StandScaler', StandardScaler()), ('pca', PCA()),
                ('estimator',
                 LGBMRegressor(boosting_type='n_estimators=138 max_depth=19 '
                                             'reg_alpha=4 reg_lambda=10 '
                                             'min_child_samples=6 '
                                             'num_leaves=160'))])>

In [ ]:
# r2
r2 = pipe.score(X_test, y_test)
print(f'r2: {r2}')

r2: 0.7903893707138792


In [ ]:
# rmse
rmse = np.sqrt(mean_squared_error(y_test, pred))

print(rmse)

4.369629487606567


In [ ]:
# モデルの保存
with open('3_pipe_lgbm.model', 'wb') as f:
  pickle.dump(pipe, f)

### 課題3の精度
- rmse: 3.752752469634236
- r2: 0.7988901108422742

In [ ]:
from sklearn import set_config
set_config(display='diagram')   
pipe

Pipeline(steps=[('StandScaler', StandardScaler()), ('pca', PCA()),
                ('estimator',
                 LGBMRegressor(boosting_type='n_estimators=138 max_depth=19 '
                                             'reg_alpha=4 reg_lambda=10 '
                                             'min_child_samples=6 '
                                             'num_leaves=160'))])

## ベイズ最適化2回目

In [ ]:
import optuna.integration.lightgbm as lgb_tune
from lightgbm import early_stopping
from sklearn.model_selection import KFold

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, shuffle=True, random_state=123)

# 標準化処理
sc = StandardScaler()
sc.fit(X_train)
X_train_sc = sc.transform(X_train)
X_test_sc = sc.transform(X_test)

trains = lgb.Dataset(data=X_train, label=y_train, feature_name='auto')
valids = lgb.Dataset(data=X_test, label=y_test, feature_name='auto')

In [ ]:
# LightGBMのパラメータ設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'random_seed':42,
    'vervose': -1 # 学習過程の表示．表示しない場合は-1，する場合は0を指定
}

best_params, history = {}, []

tuner = lgb_tune.LightGBMTunerCV(
    params,
    trains,
    folds=KFold(n_splits=7),
    verbose_eval=10,
    early_stopping_rounds=20,
)

# パラメータ探索
tuner.run()


[I 2022-01-02 23:57:01,961] A new study created in memory with name: no-name-6ebc517f-2017-4d2f-b9d1-3dadde9bb8dc
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 266
[LightGBM] [Info] Number of data points in the train set: 346, number of used features: 5
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 266
[LightGBM] [Info] Number of data points in the train set: 346, number of used features: 5
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: 

/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

feature_fraction, val_score: 3.424278:  14%|#4        | 1/7 [00:02<00:17,  2.91s/it]

...

[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000071 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 266
[LightGBM] [Info] Number of data points in the train set: 346, number of used features: 5
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019268 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 266
[LightGBM] [Info] Number of data points in the train set: 346, number of used features: 5
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: vervose
[LightGBM] [Warning] Unknown parameter: 

/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Start training from score 22.632659
[LightGBM] [Info] Start training from score 22.274567
[LightGBM] [Info] Start training from score 22.411561
[LightGBM] [Info] Start training from score 22.281556
[LightGBM] [Info] Start training from score 22.266571
[10]	cv_agg's rmse: 5.23384 + 0.809519


min_data_in_leaf, val_score: 3.247143: 100%|##########| 5/5 [00:25<00:00,  5.17s/it]

[20]	cv_agg's rmse: 3.9663 + 0.768862
[30]	cv_agg's rmse: 3.6319 + 0.785269
[40]	cv_agg's rmse: 3.44412 + 0.737304
[50]	cv_agg's rmse: 3.39953 + 0.694234
[60]	cv_agg's rmse: 3.36688 + 0.675849
[70]	cv_agg's rmse: 3.32022 + 0.628275
[80]	cv_agg's rmse: 3.30796 + 0.602969
[90]	cv_agg's rmse: 3.29846 + 0.590492
[100]	cv_agg's rmse: 3.29691 + 0.573915
[110]	cv_agg's rmse: 3.30216 + 0.558225


In [ ]:
print(tuner.best_score)
tuned_param = tuner.best_params
print(tuned_param)

3.2471433522930924
{'objective': 'regression', 'metric': 'rmse', 'random_seed': 42, 'vervose': -1, 'feature_pre_filter': False, 'lambda_l1': 0.003392256220579776, 'lambda_l2': 0.02356386679718267, 'num_leaves': 6, 'feature_fraction': 0.5, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 10}


In [ ]:
tuned_paramas = {
    'n_estimators': 100,
    'max_depth': 3,
    'reg_alpha': 0.003392256220579776,
    'reg_lambda': 0.02356386679718267,
    'min_child_samples': 5,
    'colsample _bytree': 0.5,
    'subsample': 1.0,
    'subsample_freq': 0,
    'num_leaves': 3
}

In [ ]:
# 推定器
regressor = lgb.LGBMRegressor(param_dict_to_str(tuned_paramas))

estimators = [
    ('StandScaler', scaler),
    #('feature_selection', fr),
    ('pca', pca),
    ('estimator', regressor)
]

In [ ]:
# 学習
pipe.fit(X_train, y_train)

# 予測
pred = pipe.predict(X_test)

[LightGBM] [Warning] Unknown parameter boosting_type=n_estimators=138
[LightGBM] [Warning] max_depth is set=19, max_depth=-1 will be ignored. Current value: max_depth=19
[LightGBM] [Warning] min_child_samples is set=6, min_child_samples=20 will be ignored. Current value: min_child_samples=6
[LightGBM] [Warning] num_leaves is set=160, num_leaves=31 will be ignored. Current value: num_leaves=160
[LightGBM] [Warning] reg_alpha is set=4, reg_alpha=0.0 will be ignored. Current value: reg_alpha=4
[LightGBM] [Warning] reg_lambda is set=10, reg_lambda=0.0 will be ignored. Current value: reg_lambda=10


In [ ]:
# rmse
rmse = np.sqrt(mean_squared_error(y_test, pred))

print(f'RMSE: {rmse}')

# r2
r2 = pipe.score(X_test, y_test)
print(f'r2: {r2}')

RMSE: 4.022912442186153
r2: 0.8043914094472728


In [ ]:
from sklearn import set_config
set_config(display='diagram')   
pipe

Pipeline(steps=[('StandScaler', StandardScaler()), ('pca', PCA()),
                ('estimator',
                 LGBMRegressor(boosting_type='n_estimators=138 max_depth=19 '
                                             'reg_alpha=4 reg_lambda=10 '
                                             'min_child_samples=6 '
                                             'num_leaves=160'))])

In [ ]:
from sklearn.model_selection import cross_val_score
fit_params = {'verbose': 0,  # 学習中のコマンドライン出力
              'early_stopping_rounds': 10,  # 学習時、評価指標がこの回数連続で改善しなくなった時点でストップ
              'eval_metric': 'rmse',  # early_stopping_roundsの評価指標
              'eval_set': [(X_train_sc, y_train)]  # early_stopping_roundsの評価指標算出用データ
              }

scores = cross_val_score(lgb.LGBMRegressor(), X_train_sc, y_train, cv=KFold(n_splits=7, shuffle=True, random_state=53),
                         scoring='neg_root_mean_squared_error', n_jobs=-1, fit_params=fit_params)

/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/home/bi18056/ml/venv/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stop

In [ ]:
print(f'scores={scores}')
print(f'average_score={np.mean(scores)}')

scores=[-3.55746169 -3.49258928 -3.02222467 -2.80859443 -4.83565467 -3.48991505
 -2.75413439]
average_score=-3.422939169539956


In [ ]:
# モデルの保存
with open('..//Data-science//model//lgbm.model', 'wb') as f:
  pickle.dump(pipe, f)